In [1]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO



Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import pickle

data_file = open("data_input_lucrezia.pkl",'rb')

data = pickle.loads(data_file.read())
data_file.close()

param_file =  open("pars_lucrezia.pkl",'rb')

param = pickle.loads(param_file.read())
param_file.close()


interface = cg.Interface()

In [3]:
pyro.enable_validation()



In [4]:
param["init_probs"] = 0.55
param["latent_type"] = "G"
param["lambda"] = 0.3
param["K"] = 2
param["Temperature"] = 10
param["equal_sizes_sd"] = True

In [5]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(TraceGraph_ELBO)
interface.initialize_model(data)
interface.set_model_params(param)

In [6]:
ll = interface.run(steps = 500, param_optimizer = {"lr":0.01})



Running LatentCategorical on 3768 cells with 20 segments for 500 steps and 2 modalities

Done!


ELBO: 0.437889277  : 100%|██████████| 500/500 [00:44<00:00, 11.36it/s]


In [7]:

import numpy as np

lr = interface.learned_parameters()
ICs = interface.calculate_ICs()


Computing assignment probabilities
Computing information criteria.


In [8]:
lr

{'mixture_weights_rna': array([0.8676546 , 0.13234542], dtype=float32),
 'mixture_weights_atac': array([0.94493675, 0.05506324], dtype=float32),
 'NB_size_rna': array([10.956636, 10.962325, 10.960964, 11.121573, 10.995629, 11.265587,
        11.461546, 10.960105, 10.975882, 11.055854], dtype=float32),
 'segment_factor_rna': array([18.940664, 12.26416 , 11.357063, 13.724281,  5.301623, 21.993095,
        12.306341, 10.318529, 14.44787 , 10.830394], dtype=float32),
 'NB_size_atac': array([19.711262 , 13.569104 , 11.017977 , 13.281368 , 11.480141 ,
        13.9860935, 29.878183 , 11.012781 , 11.908156 , 12.945577 ],
       dtype=float32),
 'segment_factor_atac': array([13.349896, 22.316778, 11.023577, 17.93919 ,  7.113736, 30.404491,
        19.011776, 17.869358, 16.684431, 25.170658], dtype=float32),
 'CNV_probabilities': array([[[3.6974649e-05, 3.8218008e-05, 4.4131059e-05, 8.5723426e-05,
          3.4866057e-04, 9.9944633e-01],
         [5.0204706e-01, 4.3529829e-01, 5.5994831e-02, 2.0

In [5]:
lk_atac = interface._model.likelihood(lr_t, "atac")

NameError: name 'interface' is not defined

In [43]:
from congas import log_sum_exp
lk_atac = lk_atac + torch.log(lr_t["mixture_weights_atac"]).reshape([2,1,1])

In [46]:
torch.exp(lk_atac - log_sum_exp(lk_atac))

tensor([[[9.9939e-01, 9.9989e-01, 1.4069e-04,  ..., 6.5166e-03,
          5.3351e-01, 1.5664e-07],
         [9.3591e-01, 1.6383e-01, 1.8467e-02,  ..., 6.9422e-01,
          8.2800e-01, 1.4531e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         ...,
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01]],

        [[6.1418e-04, 1.1402e-04, 9.9986e-01,  ..., 9.9348e-01,
          4.6649e-01, 1.0000e+00],
         [6.4089e-02, 8.3617e-01, 9.8153e-01,  ..., 3.0578e-01,
          1.7200e-01, 8.5469e-01],
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.9646e-01, 4.9646e-01],
         ...,
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.964

In [7]:
data["data_atac"].shape

torch.Size([10, 1895])

dict_keys(['mixture_weights_rna', 'mixture_weights_atac', 'NB_size_rna', 'segment_factor_rna', 'NB_size_atac', 'segment_factor_atac', 'CNV_probabilities', 'CNA', 'assignment_probs_rna', 'assignment_rna', 'assignment_probs_atac', 'assignment_atac'])